#  import library

In [1]:
import pandas as pd
import numpy as np
import networkx as nx 

from sklearn.ensemble import RandomForestClassifier as RFC, ExtraTreesClassifier as ETC, HistGradientBoostingClassifier as HGBC
from xgboost import XGBClassifier as XGBC
from catboost import CatBoostClassifier as CBC
from lightgbm import LGBMClassifier as LGBMC

from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import precision_score,average_precision_score,recall_score,f1_score,matthews_corrcoef,accuracy_score
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
# MCC
m_scorer = make_scorer(matthews_corrcoef, greater_is_better=True)

# AP
ap_scorer = make_scorer(average_precision_score, needs_proba=True)

# SP
def specificity_score(y_true, y_pred):
    cm_ = cm(y_true, y_pred)
    tn = cm_[0, 0]
    fp = cm_[0, 1]
    specificity = tn / (tn + fp)
    return specificity
sp_scorer = make_scorer(specificity_score, greater_is_better=True)

C:\Users\admin\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


# 10-cv

In [5]:

def read_fasta(file):
    name1=[]
    names = []
    labels = []
    seqs = []
    with open(file, 'r') as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip()
        if '>' in line:
            names.append(line)
            name1.append(line.strip('>').split()[0])
            if 'pos' in line:
                labels.append(1)
            else:
                labels.append(0)
        else:
            seqs.append(line)
    return name1,labels,seqs

def out_file(code,labels):
    data_withlabel = []
    name = []
    name.append('class')
    for j in range(code.shape[1]):
        name.append(('V' + str(j + 1)))
    # data_withlabel.append(name)
    for i in range(len(labels)):
        a = list(code[i])
        a.insert(0, labels[i])
        data_withlabel.append(a)
    df_train = pd.DataFrame(data_withlabel, columns=name)
    return df_train

# rotation operators
def rotation(a,r):
    if a == "x":
        R=np.array([[1, 0, 0],
                    [0, np.cos(r), -np.sin(r)],
                    [0, np.sin(r), np.cos(r)]])
    elif a == "y" :
        R=np.array([[np.cos(r), 0, np.sin(r)],
                    [0, 1, 0],
                    [-np.sin(r), 0, np.cos(r)]])
    elif a == "z" :
        R=np.array([[np.cos(r), -np.sin(r),0 ],
                    [np.sin(r), np.cos(r), 0],
                    [0, 0, 1]])#
    else :
        print("err!")
    return R
# S  
def s_i(stru):
    k = 0
    kk_1 = []#
    while k < len(stru):#     
        count = 1
        while k + 1 < len(stru) and stru[k] == stru[k + 1]:
            count += 1
            k += 1
        kk_1.extend([count] * count)
        k += 1

    return kk_1
# Secondary structure
group_g4=['O']
group_g1=['D','G','N','P','S']
group_g2=['A','E','H','K','L','M','Q','R']
group_g3=['C','F','I','T','V','W','Y']

keys = 'ARNDCQEGHILKMFPSTWYVO'

dc = {}
for key in keys:
    if key in group_g4:
        dc[key]='G4'
    elif key in group_g1:
        dc[key]='G1'
    elif key in group_g2:
        dc[key]='G2'   
    elif key in group_g3:
        dc[key]='G3'   
    else :
        print("err!")
# netwaork
def create_graph_from_coordinates(coordinates, threshold):
    w = len(coordinates)
    net = np.zeros((w, w))
    for p in range(w):
        for q in range(w):
            if p != q:
                d_pq = np.sqrt((coordinates[p][0] - coordinates[q][0])**2 +
                               (coordinates[p][1] - coordinates[q][1])**2 +
                               (coordinates[p][2] - coordinates[q][2])**2)
                if d_pq <= threshold:
                    net[p, q] = 1
    return nx.from_numpy_array(net)

scorings = {
    'SN': 'recall',
    'SP': sp_scorer,
    'AUC': 'roc_auc',
    'AP': ap_scorer,
    'F1': 'f1',
    'ACC': 'accuracy',
    'MCC': m_scorer}



In [6]:
name1,labels,seqs=read_fasta(r'Camellia21.fasta')
s=len(name1)
w=len(seqs[0])#   
w,s,#w sequence lengthnumbern

(21, 7824)

In [8]:


a1 = 0.4
a2 = 0.1
a3 = 0.2
Coor=[]
# Coor0=[]
c0=np.array([1, 1, 1]).reshape((3, 1))
for seq in seqs:
    sec=[dc.get(se) for se in seq]
    n_1=sec.count('G1')
    n_2=sec.count('G2')
    n_3=sec.count('G3')
    S=s_i(sec)
    coor_1=[]
    # coor_0=[[1,1,1]]
    for j in range(w):
        if j == 0:
            if sec[j]=='G1':
                coor =  rotation('x',2 * np.pi * a1 * S[j] / n_1) @ c0
            elif sec[j]=='G2':
                coor =  rotation('y',2 * np.pi * a2 * S[j] / n_2) @ c0            
            elif sec[j]=='G3':
                coor =  rotation('z',2 * np.pi * a3 * S[j] / n_3) @ c0
            elif sec[j]=='G4':
                coor = np.sqrt(3)/w + c0
            # print(j,coor)
        elif j >= 1:
            if sec[j]=='G1':
                coor =  rotation('x',2 * np.pi * a1 * S[j] / n_1) @ coor
            elif sec[j]=='G2':
                coor =  rotation('y',2 * np.pi * a2 * S[j] / n_2) @ coor            
            elif sec[j]=='G3':
                coor =  rotation('z',2 * np.pi * a3 * S[j] / n_3) @ coor
            elif sec[j]=='G4':
                coor = np.sqrt(3)/w + coor
            # print(j,coor)
        coor_1.append(list(coor)) 
        # coor_0.append(list(coor))
    # print("#####",i)
    Coor.append(coor_1)
    # Coor0.append(coor_0)


DD=[]
for i in range(s):
    DD_1=[]
    for p in range(w):
        for q in range(w):
            if p < q:
                dis=((Coor[i][p][0]-Coor[i][q][0])**2+(Coor[i][p][1]-Coor[i][q][1])**2+(Coor[i][p][2]-Coor[i][q][2])**2)**0.5
                DD_1.append(dis)
    DD.append(DD_1)
aver=np.mean(DD,axis=1)
dev=np.std(DD,axis=1)


CC = []

for i in range(s):
    G = create_graph_from_coordinates(Coor[i], aver[i])

    closeness_centrality = nx.closeness_centrality(G)
    CC.append(closeness_centrality)

CC_data = pd.DataFrame(CC)

n = 21
I = np.identity(n)

dict_code = {}
i=0
for key in keys:
    dict_code[key] = list(I[i])
    i += 1
fea = []
for seq in seqs:
    mat = []
    for mod in seq:
        mat = mat+dict_code[mod]
    fea.append(mat)
fea=np.array(fea)
data_fea=out_file(fea,labels)

bi_CC = pd.DataFrame(data_fea.iloc[:,1:].to_numpy() * np.repeat(CC_data.values,21, axis=1))


X = bi_CC.values
y = np.array(labels)
clf = ETC(random_state=0)
results = {name: round(np.mean(cross_val_score(clf, X, y, cv=10, scoring=scoring)), 4) for name, scoring in scorings.items()}
print(results)

{'SN': 0.907, 'SP': 0.8931, 'AUC': 0.9678, 'AP': 0.9682, 'F1': 0.9007, 'ACC': 0.9001, 'MCC': 0.8004}
